In [ ]:
%pip install -r requirements.txt

In [ ]:
import getpass
import os
from dotenv import load_dotenv

load_dotenv()
# GROQ_API_KEY needs to be an environment variable (create a .env file for this)
key = os.getenv(key="GROQ_API_KEY")
from langchain_groq import ChatGroq
llm = ChatGroq(model="llama3-8b-8192",api_key=key)

In [3]:
import bs4
from langchain import hub
from langchain_chroma import Chroma
from langchain_community.document_loaders import DirectoryLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [4]:
# Load and chunk the local docs
loader = DirectoryLoader('docs/', glob="**/*.txt")
docs = loader.load()

In [ ]:
embed = HuggingFaceEmbeddings()
# May want to play around with chunking sizes to make it run better
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=embed)

In [6]:
# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = ( 
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

/Users/sgupte/Documents/GitHub/THT-AI/lib/python3.12/site-packages/langsmith/client.py:234: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [9]:
input_prompt = input("Write your prompt here: ")
for chunk in rag_chain.stream(input_prompt):
    print(chunk, end="", flush=True)

The task is to create a yearbook that documents important events and milestones of the Theta Tau fraternity, along with assigning fun superlatives to deserving brothers.